In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

# Geometry and Mesh

In [2]:
square = MoveTo(0,0).Rectangle(5,1).Face()
hole = Circle((2.5,0.5), 0.2).Face()
hole.edges.name = "circle"
square.edges.Max(Y).name = "top"
square.edges.Min(Y).name = "bottom"
square.edges.Min(X).name = "inlet"
square.edges.Max(X).name = "outlet"
shape = square - hole

In [3]:
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1)).Curve(3)
#Draw(mesh)

# Deformation (Dirichlet)

In [4]:
fes_def = VectorH1(mesh, order=3, dirichlet=".*")

We create a smooth deformation by solving a Poisson equation in discrete form $ A u = f $ with $ f= 0 $.

The bilinear form $A$ includes both the FreeDofs and the fixed Dirichlet DOFs:
$$ A u = \begin{pmatrix} A_f & A_D \end{pmatrix} \begin{pmatrix} u_f \\ u_D \end{pmatrix} = 0 $$

In [ ]:
u_def, v_def = fes_def.TnT()
a_def = BilinearForm(InnerProduct(grad(u_def), grad(v_def))*dx).Assemble()
A_def_inv = a_def.mat.Inverse(freedofs=fes_def.FreeDofs())

To get $u$, we first set a fixed deformation at the boundary, `gf_def`, corresponding to $u_D$.

In [ ]:
gf_def = GridFunction(fes_def) # this will hold all deformations
gf_def_top = GridFunction(fes_def)
gf_def_top.Set((0, 2*x*(5-x)), definedon=mesh.Boundaries("top"))

From this we obtain $f_D$, using the `*`-operator (matrix-vector product):
$ f_D = -A \begin{pmatrix} 0 \\ u_D \end{pmatrix} $ \
Finally, we solve for $u_f$ by using only the `FreeDofs`: $$ u_f = A_f^{-1} \cdot f_D $$ with the `@`-operator, and add this to the total deformation.

In [ ]:
gf_def_top.vec.data -= A_def_inv@a_def.mat*gf_def_top.vec

In [ ]:
# initial values for deformations
delta_top = Parameter(-0.005)

# combine all deformations into one grid function
gf_def.vec.data = delta_top.Get() * gf_def_top.vec  # + ...

# Deformation (Neumann)

In [ ]:
mesh.SetDeformation(None)

fes_def_2 = VectorH1(mesh,order=3,dirichlet="inlet")
fes_number = NumberSpace(mesh)
X = fes_def_2 * fes_number

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, fixed